### Import library

In [1]:
""" Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP
    Goal is to predict if mutation is SNP or PD
    improve_MCC branch
    
    Total samples: 3368
    2254 PD samples
    1111 SNP samples
"""

' Example 2 is inbalanced data set; ~2200 in PD and ~1100 in SNP\n    Goal is to predict if mutation is SNP or PD\n    improve_MCC branch\n    \n    Total samples: 3368\n    2254 PD samples\n    1111 SNP samples\n'

In [2]:
""" Imports the required libraries and packages """

import pandas as pd  # Data manipulation in dataframes
import numpy as np  # Array manipulation

import random as rd # Random seed generation
import time #Time program run time
import sys

import matplotlib.pyplot as plt
from matplotlib.patches import Patch #CV visualise

# import PDB2AC

from sklearn.metrics import(
    matthews_corrcoef,  # MCC for evaluation
    balanced_accuracy_score, #hyperparameter evaluation
    f1_score,  #hyperparameter evaluation
    confusion_matrix,  # confusion matrix for classification evalutation
    classification_report #Return the F1, precision, and recall of a prediction
    )

from sklearn.model_selection import(
    train_test_split,  # Splits data frame into the training set and testing set
    GridSearchCV,  # Searches all hyperparameters
    RandomizedSearchCV, # Searches random range of hyperparameters
    GroupKFold, # K-fold CV with as groups
        )

from sklearn.utils import shuffle
from sklearn.ensemble import RandomForestClassifier #SK learn API for classificastion random forests
from sklearn.tree import DecisionTreeClassifier #Single tree decisions 
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier #allows for confidence scores to be predicted for each

np.set_printoptions(precision = 3,threshold=np.inf, suppress=True) #full array printing

### Split dataset into training and validation sets

In [3]:
def Train_Test_Split(file):
    """      
    Input:      file             Pre-processed dataset done by PDB2AC script

    Returns:    Training_Set     80% training set split
                Testing_Set      20% testing set split
                labels           Class labels for training set

    80% training and 20% testing split. Writes the data to txt files. Splits are shuffled randomly
    """
    AC_dataset = pd.read_csv(file)    
    Training_Set, Testing_Set = train_test_split(AC_dataset,train_size = 0.8)
    labels = Training_Set['dataset_pd'].astype('int32')
    Training_Set.reset_index(drop=True, inplace = True)
    Testing_Set.reset_index(drop=True, inplace = True)
    
#     Training_file = Training_Set.drop(['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'],axis=1)
#     Testing_file = Testing_Set.drop(['Binding', 'SProtFT0', 'SProtFT1', 'SProtFT2', 'SProtFT3', 'SProtFT4', 'SProtFT5', 'SProtFT6', 'SProtFT7', 'SProtFT8', 'SProtFT9', 'SProtFT10', 'SProtFT11', 'SProtFT12', 'Interface', 'Relaccess', 'Impact', 'HBonds', 'SPhobic', 'CPhilic', 'BCharge', 'SSGeom', 'Voids', 'MLargest1', 'MLargest2', 'MLargest3', 'MLargest4', 'MLargest5', 'MLargest6', 'MLargest7', 'MLargest8', 'MLargest9', 'MLargest10', 'NLargest1', 'NLargest2', 'NLargest3', 'NLargest4', 'NLargest5', 'NLargest6', 'NLargest7', 'NLargest8', 'NLargest9', 'NLargest10', 'Clash', 'Glycine', 'Proline', 'CisPro'],axis=1)

    with open('Training set.txt', 'w') as file: #Writes training data to files
        file.write(Training_Set.to_string())
    with open('Testing set.txt', 'w') as file: #Writes testing data to files
        file.write(Testing_Set.to_string())


    return Training_Set, Testing_Set, labels

### Initial evaluation

In [4]:
# def test(Initial_RFC, Input_test, Classes_test):
#     """ Input:  Input_test      Features test data
#                 Classes_test    Class label test data

#         Evaluates the training data before balancing. Random forest classifier makes prediction using the test features. True values 
#         are the class labels testing data
#     """

#     Output_pred = Initial_RFC.predict(Input_test) #Always perdict on the unseen test data, as train has been used by the estimastor
#     print(f"              **Initial Evaluation**\n")
#     print(f"Confusion Matrix:\n {confusion_matrix(Classes_test, Output_pred)}")
#     print(f"{classification_report(Classes_test, Output_pred)}\nMCC                {matthews_corrcoef(Classes_test, Output_pred)}")


## Group K-fold CV (outer loop)

In [5]:
def CV(Training_Set):
    """      
    Input:      Training_Set     80% training set split
            
    Returns:    Input_train     Training features, for each fold
                Classes_train   Ttraining classes, for each fold
                Input_val       Validating features, for each fold
                Classes_val     Validating classes, for each fold

    Group K-fold CV that maintains protein groups, attempts to preserve number of samples of each class 
    for each fold, and ensures protein groups are separated. Creates 5 folds.
    """
    
    Input_CV       = Training_Set.drop(['dataset_pd'], axis =1)         #Input features for training
    Output_CV      = Training_Set['dataset_pd'].copy().astype('int32')  #Class labels for training
    Protein_Groups = Training_Set['AC Code'].to_list()                  #List of proteins for grouping
        
    CV             = GroupKFold(n_splits = 5)                           #Only shuffles proteins in each group, not groups in fold
    
    IT_list = []
    LT_list = []
    IV_list = []
    LV_list = []
    
    for train_idx, val_idx in CV.split(Input_CV, Output_CV, Protein_Groups):
        Rd = np.random.randint(time.time())                             #Random number from 1 to time since epoch

        Input_train                        = Input_CV.loc[train_idx]
        Classes_train                      = Output_CV.loc[train_idx]
        Input_train.drop(['AC Code'], axis = 1, inplace = True)         #Group identifer not needed for training

        Input_val                          = Input_CV.loc[val_idx]
        Classes_val                        = Output_CV.loc[val_idx]
        Input_val.drop(['AC Code'], axis   = 1, inplace = True)
        
        IT_list.append(Input_train.sample(frac=1, random_state=Rd)) 
        LT_list.append(Classes_train.sample(frac=1, random_state=Rd))
        IV_list.append(Input_val.sample(frac=1, random_state=Rd))
        LV_list.append(Classes_val.sample(frac=1, random_state=Rd))
        
    with open('CV validation data.txt', 'w') as f:
        for number, fold in zip(range(len(LV_list)), LV_list):
            f.write(f"Fold: {number}\n\n{fold.to_string()}\n\n\n")

    return(IT_list, LT_list, IV_list, LV_list)


## Balancing (inner loop)

In [6]:
def find_minority_class(classData):
    """ Input:    classData  Array of class labels
    
        Returns:  minClass   The label for the minority class
                  minSize    The number of items in the minority class
                  maxSize    The number of items in the majority class

        Finds information about the inbalance in class sizes
    """
    
    Minority_count = 0
    Majority_count = 0
    for datum in classData:
        if datum == 1:
            Majority_count += 1
        elif datum == 0:
            Minority_count += 1

    minClass = 0
    minSize = Minority_count
    maxSize = Majority_count
    if Minority_count > Majority_count:
        minClass = 1
        minSize = Majority_count
        maxSize = Minority_count

    return minClass, minSize, maxSize

In [7]:
def balance(inData, classData, minClass, minSize):
    """ Input:    inData          array of input data
                  classData       array of classes assigned
                  minorityClass   class label for the minority class
                  minoritySize    size of the minority class

        Returns: array of indexes that are of interest for a balanced dataset

        Perform the actual balancing for a fold between SNPs and PDs
    """
    usedLines = [False] * len(inData) #Array of false for length of data
    for i in range(len(inData)):
        if classData[i] == minClass:
            usedLines[i] = True
            
    usedCount = 0
    while usedCount < minSize:
        i = rd.randrange(len(inData))
        if usedLines[i] == False:
            usedCount += 1
            usedLines[i] = True       

    return usedLines

In [8]:
def balance_data(inData, classData, usedLines):
    """     Input:     inData      array of input training data
                       classData   array of classes assigned to training data
                       usedLines   array of line indexes to print

            Returns:   input_balance  Array of balanced input training data
                       label_balance  Array of balanced labels training data
                       
        Create array of the input training data and classes used. Converts from array to dataframe for CV code compatibility.
        The index [i] is the identifier between the two arrays.
    """
    input_balance = []
    label_balance = []
    
    for i in range(len(inData)):
        if usedLines[i]:
            input_balance.append(inData[i])
            label_balance.append(classData[i])  
    
#     Dataframe format
#     label_balance = pd.DataFrame(label_balance, columns = ['Class'])
    
    return input_balance, label_balance

### Balance for n folds

In [9]:
def Balance_ratio(maxSize, minSize): 
    """ Input:      maxSize     The number of items in the majority class
                    minSize     The number of items in the minority class

        Returns:    BF          Number of balancing folds

        Calculate the number of balancing folds needed using ratio of majority to minority class size. Double to ensure sufficient
        majority class instances are sampled, then + 1 to make odd to allow weighted vote.
    """
    Divide = maxSize/minSize
    BF = (2 * round(Divide)) + 1 #Double ratio to nearest integer
    return BF

In [10]:
def Balance_Folds(BF, inData, classData, minClass, minSize):
    """ Input:      BF                Number of balancing folds needed
                    usedLines         Array of line indexes to print
                    input_balance     Dataframe of balanced training data
                    label_balance
                    
        Returns:    Input_folds       List of balanced arrays of training data
                    Output_folds

        Perform the balance_data() function n number of balancing fold times. Return lists for feature data and labels
        where each item is the output of balance_data()
    """
    Input_folds = []
    Output_folds = []
    fold_df = []

    for i in range(BF):
        usedLines = balance(inData, classData, minClass, minSize)
        input_balance, label_balance = balance_data(inData, classData, usedLines)
        
        Input_folds.append(input_balance)
        Output_folds.append(label_balance)
        
        df = pd.DataFrame(input_balance)
        fold_df.append(df)
        
    with open('Balanced training data.txt', 'w') as f:
        for number, fold in zip(range(BF), fold_df):
            f.write(f"Fold: {number}\n\n{fold}\n\n\n")
            
    return Input_folds, Output_folds

### RFC hyperparameter tuning

In [11]:
#  def Hyperparameter(BF, Input_folds, Output_folds):
#     """ Input:      BF                Number of balancing folds needed
#                     Input_folds       List of 5 balanced arrays of training data
#                     Output_folds      List of 5 balanced arrays of training data's labels

#         Returns:    BF_RFC_HP         List of optimized hyperparameters for each RFC

#         Perform RandomSearchCV on each RFC to optimize number of trees, max depth and max samples
#     """  
#     estimator = RandomForestClassifier()
#     param_grid = {
#                 'n_estimators':np.arange(50,500,50),
#                 'max_depth': np.arange(2, 10, 2),
#                 'max_samples': np.arange(0.2, 1.2, 0.2)
#                   }
#     BF_RFC_HP = []

#     for i in range(BF):
#         HPtuning = RandomizedSearchCV(
#             estimator,
#             param_grid, 
#             scoring = 'balanced_accuracy',
#             cv = 10,
#             n_jobs = 6, #how many cores to run in parallel
#             verbose = 2
#             ).fit(Input_folds[i], Output_folds[i].ravel())
#         BF_RFC_HP.append(HPtuning.best_params_)
    
#     return(BF_RFC_HP)

### Train RFC on the trainings folds

In [12]:
def BF_fitting(Input_folds, Output_folds): 
    """ Input:
                    Input_train_list      List of 5 balanced arrays for training data
                    Classes_train_list    List of 5 balanced arrays of training data labels
        Returns:    BF_RFC                List of RFC's trained on data in each balancing fold

        Create RFC model that returns probability predictions for each fold, using output of Balance_Folds() as training data
    """    
    BF_RFC = []
    for i in range(BF):
        BF_RFC.append(RandomForestClassifier(verbose = 1, n_estimators = 1000)) #Generates a RFC for each fold's training data
        BF_RFC[i].fit(Input_folds[i], Output_folds[i]) #Fits the RFC to each folds' training data
        
    return BF_RFC

#### Validate each RFC on validation set, for each fold

In [13]:
def BF_validate(BF_RFC, ValData):
    """ Input:  BF_RFC          List of RFC's trained on data in each balancing fold
                Input_val_list  Unseen validation data fold from CV fold
                
        Returns:Prob_matrix     List of arrays. Each item is 2D matrix where the 1st dimension is each subset in balancing fold, 
                                2nd dimension is predicted probability
    
        Test the trained RFCs on the test set, then for every instance, outputs the predicted probability for each class
    """
    
    Prob_matrix = [] #Empty list
    for i in range(len(BF_RFC)):
        Prob = BF_RFC[i].predict_proba(ValData.values)
        Prob_matrix.append(Prob)   
            
    with open('Validation fold probabilities.txt', 'w') as f:
        for number, line in zip(range(BF), Prob_matrix ):
            f.write(f"Fold: {number}\n\n   SNP    PD\n{line}\n\n\n")

    return Prob_matrix

In [27]:
(Prob_matrix[0][:,0][0])

0.597

### Weighted voting

In [14]:
def Weighted_Vote(Prob_matrix, BF):
    """ Input:      Prob_matrix     List of arrays. 2D matrix where the 1st dimension is each subset in balancing fold, 
                                    2nd dimension is predicted probability
                    BF              Number of balancing folds

        Returns:    Final_vote      Weighted vote classification

        Calculate the final weighted vote using confidence scores (Sc). Binary classification formula Sc = 2|S0 - 0.5|
    """
    Sc_PD = []
    Sc_SNP = []
    for i in range(BF):
        Sc_PD.append(2* (Prob_matrix[i][:,1] - 0.5)) #Confidence scores for PD, for each fold
        Sc_SNP.append(2*(Prob_matrix[i][:,0] - 0.5)) #Confidence scores for SNP, for each fold

    Sum_PD = np.sum(Sc_PD, axis = 0) #Sum of all PD confidence scores. 1D Array
    Sum_SNP = np.sum(Sc_SNP, axis = 0) #Sum of all SNP confidence scores. 1D Array     
    print(Sum_PD)
    Vote_arr = [] #Empty list

    for i in range(len(Sum_PD)):
        if Sum_PD[i] >= Sum_SNP[i]:
            Vote_arr.append([1]) #Append PD classifications to list
        elif Sum_SNP[i] > Sum_PD[i]:
            Vote_arr.append([0]) #Append SNP classifications to list

        Final_vote = np.stack(Vote_arr) #Converts list of arrays to a 2D array
        Final_vote = Final_vote.ravel() #Flattens 2D array to 1D array

    return(Final_vote, Sum_PD, Sum_SNP) #Returns the final confidence scores


In [29]:
    T = 0.45

    for fold in range(BF): # Calculates SNP Sc all instances in each fold
        Sc_SNP = []
        for value in range(len(Prob_matrix[fold][:,0])):
            S0 = Prob_matrix[fold][:,0][value] #each SNP value in prob matrix fold
            if S0 < T:
                Sc = (T - S0)/T
            elif S0 >= T:
                Sc = (S0 - T)/(1 - T)        
            Sc_SNP.append(Sc)   

    for fold in range(BF):# Calculates PD Sc all instances in each fold
        Sc_PD = []
        for value in range(len(Prob_matrix[fold][:,1])):
            S0 = Prob_matrix[fold][:,1][value] #each PD value in prob matrix fold
            if S0 < T:
                Sc = (T - S0)/T
            elif S0 >= T:
                Sc = (S0 - T)/(1 - T)        
            Sc_PD.append(Sc) 
            
Sc_SNP

[0.2054545454545453,
 0.061818181818181765,
 0.5509090909090909,
 0.1018181818181818,
 0.20181818181818187,
 0.7618181818181817,
 0.21111111111111117,
 0.23272727272727262,
 0.0745454545454545,
 0.7363636363636362,
 0.4127272727272728,
 0.6418181818181818,
 0.2618181818181817,
 0.09555555555555564,
 0.36363636363636365,
 0.14181818181818184,
 0.5781818181818181,
 0.05454545454545449,
 0.7418181818181817,
 0.7272727272727272,
 0.031111111111111138,
 0.11777777777777776,
 0.0036363636363636394,
 0.16000000000000003,
 0.7399999999999999,
 0.013333333333333345,
 0.3709090909090909,
 0.6666666666666667,
 0.1654545454545455,
 0.7927272727272726,
 0.13333333333333333,
 0.7,
 0.0036363636363636394,
 0.47111111111111115,
 0.3155555555555556,
 0.0,
 0.06727272727272722,
 0.07090909090909087,
 0.5399999999999999,
 0.509090909090909,
 0.4636363636363635,
 0.3244444444444445,
 0.06666666666666672,
 0.0745454545454545,
 0.33999999999999997,
 0.21090909090909077,
 0.25818181818181807,
 0.322222222222

### Final confidence

In [16]:
def Final_score(Sum_PD, Sum_SNP, BF):
    """ Input:      Sum_PD      Sum of confidence score for PD predictions
                    Sum_SNP     Sum of confidence score for SNP predictions

        Returns:    S_out       Final confidence score

        Calculate the final confidence score
    """
    
    S_Out = np.abs((Sum_PD - Sum_SNP) /(BF*2))
    np.savetxt('S_out.txt', S_Out, "%.3f")
    
    return S_Out


In [17]:
def evalutation(Vallabel, Final_vote, S_Out):
    """ Input:      Classes_val_list   Validation label data
                    Final_vote         Weighted vote classification

        Evaluation metrics from RFC on validation data with
    """
    Output_pred = Final_vote
    print(f"              ***Final Evaluation***\n")
    print(f"Confusion Matrix:\n {confusion_matrix(Vallabel, Output_pred)}")
    print(f"{classification_report(Vallabel, Output_pred)}\nMCC                {matthews_corrcoef(Vallabel, Output_pred)}")

    print(f"See file 'Classification.txt' for final classifications and confidence scores")
    np.savetxt('Classification.txt',
           np.column_stack([Final_vote, S_Out]),
           fmt = ["%.0f","%.3f"],
           delimiter ="      ",
           header = "Final classifications and confidence scores\n\n"
          )


### Main Program

In [18]:
file = "AC_dataset.csv"
Training_Set, Testing_Set, labels  = Train_Test_Split(file)
IT_list, LT_list, IV_list, LV_list = CV(Training_Set)

for i in range(len(IT_list)): #For every CV fold
    classData                   = LT_list[i].to_numpy()
    inData                      = IT_list[i].to_numpy()
    ValData                     = IV_list[i]
    Vallabel                    = LV_list[i]

    minClass, minSize, maxSize  = find_minority_class(classData)
    BF                          = Balance_ratio(maxSize, minSize)
    Input_folds, Output_folds   = Balance_Folds(BF, inData, classData, minClass, minSize) #balance() and balance_data() functions are called under this

    BF_RFC                      = BF_fitting(Input_folds, Output_folds)
    Prob_matrix                 = BF_validate(BF_RFC, ValData)

    Final_vote, Sum_PD, Sum_SNP = Weighted_Vote(Prob_matrix, BF)
    S_Out                       = Final_score(Sum_PD, Sum_SNP, BF)

    evalutation(Vallabel, Final_vote, S_Out)

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

[ 1.202  2.344 -0.97  -2.072  0.066 -1.796  2.048 -1.53  -1.698 -2.704
 -0.906  2.21  -0.93   1.712 -0.76   1.226 -0.92   1.926  0.702 -2.092
 -3.054 -1.662  0.576  2.302 -0.444 -0.392 -1.218 -1.71   2.94   1.248
 -1.766  0.424 -1.188  0.018  1.956 -1.456 -2.482 -1.564 -2.752  0.408
 -1.362 -2.614 -1.672 -1.552  3.624  3.446 -2.294  0.918 -1.112 -2.632
 -2.806 -1.484  0.426 -3.014 -0.832  0.456  1.424 -4.078 -0.866 -0.724
 -1.446  1.848 -1.136 -0.832  0.508  0.156 -2.396 -3.582 -1.034 -0.374
 -0.058 -0.464  0.562  0.754  1.254  0.21  -1.604 -0.312  1.346  1.05
 -2.496 -1.934 -0.624  0.77  -0.736  1.808 -0.026  2.256  0.494  2.608
  2.84   2.084 -1.86  -0.006 -2.898  0.19   0.228 -2.38  -3.778  2.758
 -3.008 -1.016 -2.024  0.148 -1.992  0.514 -2.296  0.91  -0.644  1.078
 -0.836 -3.724  1.446 -3.774 -0.312  2.692  1.124 -2.188 -0.322 -0.706
 -2.58  -4.264  0.712 -1.268 -0.668 -3.558 -0.666  2.432 -3.012 -3.068
  1.734 -0.376  0.198  2.902  1.036  2.06  -2.84  -1.706 -1.98   1.34
 -1.35  

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

[-3.402 -1.348  0.13  -1.106  1.556 -3.252  0.27  -1.74  -0.5   -3.404
 -2.228  1.246 -0.404  0.244 -0.728 -3.616 -0.806 -2.616 -0.152 -0.188
  0.524  1.318  0.668 -1.024  0.766 -2.982  0.116 -2.946  2.228  1.794
 -2.924  3.234 -0.036 -1.362  2.786 -2.452 -0.2    0.618 -2.632 -1.956
 -1.552 -2.446  1.114  1.372 -3.148  1.934  1.476  1.608 -1.904 -2.018
 -0.218 -3.156 -3.122  1.952  1.556  1.708 -2.158 -3.656  0.86   1.646
 -3.152 -1.756  0.92   0.588 -2.828 -0.978 -0.1    2.864  2.308 -1.014
  0.554  0.754 -0.606 -1.926  2.092  2.546 -1.85  -0.718 -0.068  2.41
 -0.7   -3.24  -0.35  -1.306 -2.936 -2.642 -1.234 -1.322 -0.56  -2.678
 -1.24  -1.424  2.086 -0.636 -1.66  -0.538  1.34   2.66  -3.236  0.138
 -2.258 -2.016  2.258  0.55  -2.746 -1.436 -2.01  -1.124 -0.846 -2.404
 -2.986  2.98  -3.866  2.274 -2.58   0.194  1.436  2.942  2.31  -0.362
  2.142 -1.948 -0.312 -0.316  1.008  0.218 -1.502  2.132 -1.922  1.208
  0.088 -0.75  -3.528  2.634  1.272  1.364 -1.602 -1.708 -0.816 -0.588
 -1.04 

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.7s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

[-3.516  1.102 -1.148 -2.27  -0.056  0.768 -1.464  0.282 -2.462 -2.246
 -2.508 -1.986  0.766  3.344  0.832  1.358 -1.422 -0.242 -2.238  0.67
 -0.814 -0.88   0.246 -0.094  1.606 -1.724 -1.394  2.78   2.414 -0.99
  0.518 -1.906 -1.184 -2.056  0.91  -1.642 -1.116 -3.068 -1.644 -0.554
 -1.594  1.424 -1.428  1.08   1.516  1.37   0.166 -0.488 -1.558 -1.066
 -1.266 -2.504  0.916 -2.104  2.896 -2.488 -3.874  0.914  1.444  1.34
  2.744  0.33  -1.082 -0.74   0.034 -0.19   1.532  2.66  -0.892  1.086
  0.138  0.054  1.156 -3.812  0.784  0.702  0.452 -0.424 -0.918 -1.31
 -3.978  2.118 -0.724 -0.944  1.826 -2.29   0.156  0.884 -1.764 -1.482
 -0.436 -1.794 -0.562 -0.996  2.946 -2.824 -1.256 -2.96  -1.42  -1.962
  0.236 -0.074  0.408  1.138  2.608 -1.276 -0.346 -1.614 -1.28   0.402
 -2.646  2.75   1.38   0.318 -1.542 -2.438 -3.438 -0.238  1.83  -0.938
 -3.11  -1.14   2.068  0.782 -0.796  0.524 -1.468  0.348 -1.88  -1.502
 -0.73  -1.058 -1.352 -1.422 -0.896  2.308 -2.106  0.812 -1.768 -0.644
  3.138 -2

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

[ 1.13  -3.902  1.702  0.612  1.79  -0.196 -1.656 -0.424 -2.518 -3.772
  2.35   1.862 -3.136 -2.31   3.246 -3.15  -1.19   2.352 -2.37   0.798
 -2.028  2.132 -2.468 -0.012 -1.428  0.774  0.662 -1.336 -1.944 -2.756
  1.018  2.91   0.332 -1.032  1.81  -0.576 -0.218  2.156 -0.23   1.94
 -0.85   1.4    0.352  1.09  -4.228 -2.024  0.18   3.464  3.044 -1.48
  1.316 -2.178  0.912  1.158  1.58  -4.064 -2.232  0.474 -2.366 -2.366
 -0.342 -3.032  0.448 -2.73  -1.3   -0.4    2.82   3.192 -2.906  0.664
  1.388  3.69   0.978 -2.508 -1.742  0.776 -2.248 -1.34  -0.764 -1.042
 -2.754 -2.75   2.504 -4.232  1.52  -2.262  0.07  -0.524 -2.992  1.652
  1.028  2.486 -3.276  1.026 -2.152  0.124  0.858 -2.756  0.404  1.606
 -0.094 -2.178  0.928 -1.354 -1.224 -2.018  1.64  -2.36   0.432 -3.83
  0.644 -1.868  0.832 -0.318 -0.302 -1.812 -3.426  1.364 -2.246 -2.038
  1.224  1.71   3.254 -2.104 -0.62  -2.294  2.682 -1.064  1.174 -0.05
 -2.142 -2.878 -0.126 -2.138  0.544  1.912 -0.44  -1.394 -0.86  -1.466
  0.438  1

[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.4s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.6s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    2.5s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done 1000 out of 1000 | elapsed:    0.0s finished
[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[

[-0.926  0.426 -2.712 -0.192 -1.012 -3.338  1.426 -0.71  -0.128 -3.528
 -1.786 -3.168 -1.418  1.034 -1.9   -0.552 -2.812 -0.044 -3.26  -2.956
  0.014  1.442 -0.084 -0.578 -3.24   1.628 -1.338  3.574 -0.866 -3.584
  1.118  3.556  0.768  2.632  1.606  0.288  0.894  0.212 -2.644 -2.352
 -2.604  1.81   0.374 -0.042 -2.188 -0.282 -0.806  1.48  -0.412 -3.76
  1.372  0.62   1.972 -1.498 -1.37   3.888 -3.504  2.322 -0.644 -0.234
  0.51  -0.438  1.62  -2.522 -4.026 -1.088 -0.38  -1.508  1.038 -1.618
 -2.286 -1.192 -0.246 -0.444  1.922  0.31   0.07  -3.012 -2.49  -1.674
 -0.158 -1.566 -0.922 -2.012 -3.01   0.016 -1.6   -1.704 -4.06   1.066
 -2.668 -1.798  0.202 -1.382  1.104 -2.722 -3.102 -2.216  0.494  1.026
 -1.592  0.202  0.546 -2.724  1.396 -2.638  2.612  1.652  2.652 -1.098
  1.906 -0.724 -1.65   1.222 -3.952  2.864  0.248  1.942 -0.578  0.536
 -1.9    1.78  -0.77   0.374  0.578 -0.18  -3.176  0.206 -1.806  1.92
 -3.446 -1.66   1.15  -1.328 -2.712 -2.116  2.188 -2.88   1.884  1.44
 -1.702 -